# Quantitative Value Strategy

"Value investing" means investing in the stoks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portifolio of these 50 stocks.

# Library imports

In [2]:
from scipy.stats import percentileofscore as score
from scipy import stats
import pandas as pd
import numpy as np 
import xlsxwriter 
import requests
import math

# Importing Our List of Stocks and getting API Token


In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

data

{'companyName': 'Apple Inc',
 'marketcap': 2279834785759,
 'week52high': 189.43,
 'week52low': 130.03,
 'week52highSplitAdjustOnly': 187.22,
 'week52lowSplitAdjustOnly': 132.5,
 'week52change': -0.02664256240022324,
 'sharesOutstanding': 16170549081,
 'float': 0,
 'avg10Volume': 117572928,
 'avg30Volume': 102311556,
 'day200MovingAvg': 160.5,
 'day50MovingAvg': 162.01,
 'employees': 150064,
 'ttmEPS': 6.28,
 'ttmDividendRate': 0.9261351061471336,
 'dividendYield': 0.006706943628063333,
 'nextDividendDate': '',
 'exDividendDate': '2022-07-27',
 'nextEarningsDate': '2022-10-26',
 'peRatio': 23.089026259355506,
 'beta': 1.288292423273083,
 'maxChangePercent': 52.67773687830902,
 'year5ChangePercent': 2.851958368471667,
 'year2ChangePercent': 0.2025107251073653,
 'year1ChangePercent': -0.026049269294579305,
 'ytdChangePercent': -0.2240362070531773,
 'month6ChangePercent': -0.21029277069825372,
 'month3ChangePercent': -0.004056405918694425,
 'month1ChangePercent': -0.12901692906407522,
 'da

# Executing a batch API Call & Building our Dataframe

In [4]:
#1.Split the list of tickers in sublists.

#2. Using chuncks function
#2.1. Source: https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks

#3. See batch requests in IEX API documentation

def chunks(lst,n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

In [5]:
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    
    symbol_strings.append(','.join(symbol_groups[i]))
    #print (symbol_strings[i])
    
my_columns = ['Ticker','Company Name', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string} &types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
    
        final_dataframe= final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['stats']['companyName'],
            data[symbol]['price'],
            data[symbol]['stats']['peRatio'],
            'N/A'
        ],
        index = my_columns),
        ignore_index = True)
        
final_dataframe.head(50)

,Ticker,Company Name,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,AAP,Advance Auto Parts Inc,156.53,17.668959,N/A
1,AAPL,Apple Inc,143.70,23.246471,N/A
2,ABBV,Abbvie Inc,136.58,19.151486,N/A
3,ABC,Amerisource Bergen Corp.,142.01,15.802131,N/A
4,ABMD,Abiomed Inc.,249.59,53.535903,N/A
5,ABT,Abbott Laboratories,100.90,20.001903,N/A
6,ACN,Accenture plc,269.30,26.035149,N/A
7,ADBE,Adobe Inc,284.60,27.409016,N/A
8,ADI,Analog Devices Inc.,146.05,38.062337,N/A
9,ADM,Archer Daniels Midland Co.,80.53,13.126478,N/A


# Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 
Since the goal of this strategy is to identify the 50 best value stocks from our universe, our 
next step is ro remove glamour stocks from the DataFrame. 

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the
top 50. 

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = True, inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]

#Returning the top 10
final_dataframe = final_dataframe[:10]
final_dataframe.reset_index(inplace=True)
final_dataframe.drop('index', axis = 1, inplace = True)

final_dataframe

,Ticker,Company Name,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,AIG,American International Group Inc,49.02,2.944864,N/A
1,APA,APA Corporation,34.20,3.726560,N/A
2,AIV,Apartment Investment & Management Co.,7.50,4.653875,N/A
3,CE,Celanese Corp,93.69,5.035795,N/A
4,C,Citigroup Inc,42.79,5.223369,N/A
5,BEN,"Franklin Resources, Inc.",22.31,6.591147,N/A
6,AIZ,Assurant Inc,151.02,6.658682,N/A
7,CF,CF Industries Holdings Inc,96.91,7.592514,N/A
8,BBY,Best Buy Co. Inc.,63.75,8.288937,N/A
9,AFL,Aflac Inc.,57.40,8.477458,N/A


# Calculating the number os shares to buy

In [9]:
portifolio_input = 100000
position_size = float(portifolio_input)/len(final_dataframe.index)

for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])

final_dataframe

,Ticker,Company Name,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,AIG,American International Group Inc,49.02,2.944864,203
1,APA,APA Corporation,34.20,3.726560,292
2,AIV,Apartment Investment & Management Co.,7.50,4.653875,1333
3,CE,Celanese Corp,93.69,5.035795,106
4,C,Citigroup Inc,42.79,5.223369,233
5,BEN,"Franklin Resources, Inc.",22.31,6.591147,448
6,AIZ,Assurant Inc,151.02,6.658682,66
7,CF,CF Industries Holdings Inc,96.91,7.592514,103
8,BBY,Best Buy Co. Inc.,63.75,8.288937,156
9,AFL,Aflac Inc.,57.40,8.477458,174


# Build a Better (and More Realistic) Value Strategy

Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings. Similary, stocks that buyback their own shares are dificult to value using the price-to-book ratio.

Investors typically use a composite basket of valuation metrics to build robust quantitative value strategies.

In this section, we will filter for stocks with the lowest percentiles on the following metrics:

    1. Price-to-earnings ratio (Equivalent in portuguese: P/L - Preço sobre lucro)
    2. Price-to-book ratio (Equivalent in portuguese: P/VP - Preço sobre Valor Patrimonial)
    3. Price-to-sales ratio (Equivalent in portuguese: P/S - Preço sobre vendas
    4. Enterprise Value divided by earnings before interest, taxes, depreciation, and amortization (EV/EBITDA)
    5. Enterprise Value divided by gross profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data.

In [8]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol} &types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

print(data['AAPL']['advanced-stats']['priceToBook'])


# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio 
pb_ratio = data['AAPL']['advanced-stats']['priceToBook']

# Price-to-sales ratio 
ps_ratio = data['AAPL']['advanced-stats']['priceToSales']

# Enterprise value divided by earnings before interest, taxes, depreciation, and amortization (EV/EBITDA)
enterprise_value = data['AAPL']['advanced-stats']['enterpriseValue']
ebitda = data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# Enterprise value divided by Gross Profit (EV/GP)
gross_profit = data['AAPL']['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit


48.6


In [9]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


# Continue on Project 4...